# Prototipo de Agente Orquestador de MLOps con LangGraph

## 1. Instalación e Importaciones

Instalamos las dependencias y luego importamos los módulos necesarios. Importamos nuestras funciones de nodos desde el archivo `utils.py`. 

In [ ]:
import sys
import os
from langgraph.graph import StateGraph, END

# Añadimos el directorio actual al path para poder importar nuestro módulo de utilidades
sys.path.append(os.path.abspath('..'))

# Importamos el estado y todas las funciones de los nodos desde utils.py
from notebooks.utils import *


## 2. Definición de las Aristas Condicionales

Las aristas condicionales nos permiten crear ramas en nuestro grafo. La primera (`data_found_check`) decide si continuar el flujo o detenerse si no hay datos nuevos.

In [ ]:
def data_found_check(state: MLOpsState) -> str:
    print("--- (EDGE) ¿Se encontraron datos nuevos? ---")
    if state.get("new_data_path") == "NO_NEW_DATA":
        print("--> No. Finalizando ejecución.")
        return "end"
    else:
        print("--> Sí. Procediendo a validar datos.")
        return "continue"

def should_train(state: MLOpsState) -> str:
    print("--- (EDGE) ¿Datos de buena calidad? ---")
    if state.get("failure_analysis_report"):
        print("--> No. Alertando a humano.")
        return "alert_human"
    else:
        print("--> Sí. Procediendo a entrenamiento.")
        return "run_training_job"

def should_deploy(state: MLOpsState) -> str:
    print("--- (EDGE) ¿Decisión de despliegue? ---")
    if state.get("deployment_decision") == "APPROVE":
        print("--> APROBADO. Desplegando a producción.")
        return "deploy_to_production"
    else:
        print("--> RECHAZADO. Alertando a humano.")
        return "alert_human"


## 3. Construcción y Compilación del Grafo

Ensamblamos todos los nodos y aristas importados para construir el grafo ejecutable.

In [ ]:
workflow = StateGraph(MLOpsState)

# Añadir nodos
workflow.add_node("check_for_new_data", check_for_new_data)
workflow.add_node("validate_data", validate_data)
workflow.add_node("run_training_job", run_training_job)
workflow.add_node("analyze_performance", analyze_performance)
workflow.add_node("deploy_to_production", deploy_to_production)
workflow.add_node("alert_human", alert_human)

# Definir punto de entrada
workflow.set_entry_point("check_for_new_data")

# Añadir aristas
workflow.add_conditional_edges("check_for_new_data", data_found_check, {"continue": "validate_data", "end": END})
workflow.add_conditional_edges("validate_data", should_train, {"run_training_job": "run_training_job", "alert_human": "alert_human"})
workflow.add_edge("run_training_job", "analyze_performance")
workflow.add_conditional_edges("analyze_performance", should_deploy, {"deploy_to_production": "deploy_to_production", "alert_human": "alert_human"})

# Definir puntos finales
workflow.add_edge("deploy_to_production", END)
workflow.add_edge("alert_human", END)

# Compilar el grafo
app = workflow.compile()
print("Grafo compilado y listo para usar.")

## 4. Ejecución del Grafo

Ahora podemos invocar el grafo. Si no tienes configurado un bucket, el primer nodo fallará de forma segura y terminará el ciclo. Si creas un bucket y una carpeta `new`, puedes probar el flujo completo.

In [ ]:
# Para probar, puedes crear un archivo .env en la raíz del proyecto con tus credenciales:
# S3_ENDPOINT_URL=https://<tu-endpoint>
# S3_ACCESS_KEY=<tu-access-key>
# S3_SECRET_KEY=<tu-secret-key>

# from dotenv import load_dotenv
# load_dotenv()

initial_state = {"model_version": 2} 
final_state = app.invoke(initial_state)

print("--- ESTADO FINAL DEL GRAFO ---")
print(final_state)